In [1]:
import networkx as nx
import numpy as np
import functools
import random

In [2]:
from optiLib.problem import NetGraph
from optiLib.problem import TaskGraph

def generate_adjacency_matrix(rows, cols):
    num_nodes = rows * cols
    adjacency_matrix = [[0 for _ in range(num_nodes)] for _ in range(num_nodes)]

    for row in range(rows):
        for col in range(cols):
            node_index = row * cols + col
            if col > 0:  # узел слева
                adjacency_matrix[node_index][node_index - 1] = 1
            if col < cols - 1:  # узел справа
                adjacency_matrix[node_index][node_index + 1] = 1
            if row > 0:  # узел сверху
                adjacency_matrix[node_index][node_index - cols] = 1
            if row < rows - 1:  # узел снизу
                adjacency_matrix[node_index][node_index + cols] = 1

    return adjacency_matrix


martx = [
    [0, 1000, 100, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 50, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 100, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 100, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50], 
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
]

net  = NetGraph(generate_adjacency_matrix(3, 3), net_power=(1000, 2500), net_power_arr=[5000, 500, 2000, 1000, 3000, 2000, 300, 1500, 1000], e0=(10, 50), emax=(70,100))
task = TaskGraph(martx, w_arr=[500, 1000, 1000, 1000, 500, 500, 500, 200, 200, 200, 100, 100, 100])

In [3]:
import functools
import networkx as nx
import numpy as np

@functools.lru_cache
def create_paths(vector, task_edges, network_graph):
    RANDOM_SEED = 1
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)

    paths = {}
    
    for start, end in task_edges:
        if vector[start] == vector[end]:
            paths[str(start) + str(end)] = [vector[start]]
        else:
            paths[str(start) + str(end)] = random.choice(list(nx.all_shortest_paths(network_graph, vector[start], vector[end])))
    return paths

@functools.lru_cache
def network_status_calculation(vector, problem):
    RANDOM_SEED = 1
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    
    T_lim = problem.t_lim
    vector_tuple = tuple(vector) 
    task_edges = tuple((start, end) for start, end, _ in nx.to_edgelist(problem.task_graph.graph))
    network_graph = problem.network_graph.graph
    
    paths = create_paths(vector_tuple, task_edges, network_graph)
    
    net_number = network_graph.number_of_nodes()
    task_number = problem.task_graph.graph.number_of_nodes()

    W = [0] * net_number
    v_task_to_node = [0] * net_number
    v_sent_to_node = [0] * net_number
    v_reseive_to_node = [0] * net_number
    
    for start, end, weight in nx.to_edgelist(problem.task_graph.graph):
        path_key = str(start) + str(end)
        path = paths[path_key]
        
        if len(path) > 1:
            W[path[0]] += problem.task_graph.operations[start].w + weight['weight']
            v_sent_to_node[path[0]] += weight['weight']
            v_task_to_node[path[0]] += problem.task_graph.operations[start].w
            
            W[path[-1]] += problem.task_graph.operations[end].w + weight['weight']
            v_reseive_to_node[path[-1]] += weight['weight']
            v_task_to_node[path[-1]] += problem.task_graph.operations[end].w
            
            for node in path[1:-1]:
                W[node] += 2 * weight['weight']
                v_sent_to_node[node] += weight['weight']
                v_reseive_to_node[node] += weight['weight']
        else:
            W[vector[start]] += problem.task_graph.operations[start].w
            v_task_to_node[vector[start]] += problem.task_graph.operations[start].w
            
            W[vector[end]] += problem.task_graph.operations[end].w
            v_task_to_node[vector[end]] += problem.task_graph.operations[end].w
    
    vector_tuple = tuple(vector) 
    
    arr = sorted(nx.all_simple_paths(problem.task_graph.graph, 0, task_number - 1), key=len, reverse=True)
    
    T_task = [0] * task_number
    len_first_path = len(arr[0])
    time_per_task = T_lim / len_first_path
    
    for task in arr[0]:
        T_task[task] = time_per_task
    
    for path_task in arr[1:]:
        sum_t = sum(T_task[task] for task in path_task)
        temp = [task for task in path_task if T_task[task] == 0]
        
        if temp:  # если нашлись задачи без оценки времени
            time_remaining = max(0, T_lim - sum_t)
            time_per_task_temp = time_remaining / len(temp)
            for task in temp:
                T_task[task] = time_per_task_temp
    
    T_NET = [0] * net_number

    for start, end, _ in nx.to_edgelist(problem.task_graph.graph):
        path_net = paths[str(start) + str(end)]
        if len(path_net) == 1:
            T_NET[path_net[-1]] = min(T_NET[path_net[-1]] or time_per_task, T_task[start], T_task[end])
        else:
            T_NET[path_net[-1]] = min(T_NET[path_net[-1]] or T_task[end], T_task[end])
            time_start_share = T_task[start] / (len(path_net) - 1)
            for node in path_net[:-1]:
                T_NET[node] = min(T_NET[node] or time_start_share, time_start_share)
    
    special_t = T_lim / len_first_path
    result = [[] for _ in range(net_number)]
    for i, val in enumerate(vector):
        result[val].append(i)
    
    for node in range(net_number):
        common_tasks = set(result[node]) & set(arr[0])
        if len(common_tasks) > 1:
            T_NET[node] = special_t * len(common_tasks)
    
    D = [W[i] / (problem.network_graph.nodes[i].p * T_NET[i]) if T_NET[i] != 0 else 0 for i in range(net_number)]
    return {"w":W, "v_task_to_node":v_task_to_node, "v_sent_to_node":v_sent_to_node, "v_reseive_to_node":v_reseive_to_node, "D":D, "T_NET":T_NET, "T_task":T_task}

@functools.lru_cache
def status(vector, problem):
    RANDOM_SEED = 1
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    
    vector_tuple = tuple(vector) 

    task_graph = problem.task_graph.graph
    task_edges = tuple((start, end) for start, end, _ in nx.to_edgelist(task_graph))

    network_graph = problem.network_graph.graph
    
    paths = create_paths(vector_tuple, task_edges, network_graph)
    
    net_number = network_graph.number_of_nodes()
    task_number = task_graph.number_of_nodes()

    time_task = [problem.task_graph.operations[task_id].w/problem.network_graph.nodes[node_id].p for task_id, node_id in enumerate(vector_tuple)]
    time_node = [0] * net_number
    for task_id, node_id in enumerate(vector_tuple):
        time_node[node_id] += time_task[task_id]


    edge_hop = {key: len(value) - 1 for key, value in paths.items()}
    time_edge = {(start, end): edge_hop[str(start) + str(end)] * weight['weight'] / problem.net_speed for start, end, weight in nx.to_edgelist(problem.task_graph.graph)}

    H = task_graph.copy()

    # Добавляем атрибут времени выполнения каждой задаче
    nx.set_node_attributes(H, time_task, 'execution_time')

    # Добавляем атрибут времени передачи данных каждому ребру
    nx.set_edge_attributes(H, time_edge, 'transfer_time')

     # Функция для расчета самого длинного пути до узла
    def longest_path_to_node(node):
        predecessors = list(H.predecessors(node))
        if not predecessors:
            return time_task[node]
        
        max_path = 0
        for pred in predecessors:
            pred_path = longest_path_to_node(pred)
            edge_time = H[pred][node]['transfer_time']
            path_length = pred_path + edge_time + time_task[node]
            max_path = max(max_path, path_length)
        
        return max_path

    # Находим узлы без исходящих рёбер (конечные задачи)
    end_nodes = [node for node in H.nodes() if H.out_degree(node) == 0]

    # Вычисляем максимальное время выполнения
    max_execution_time = max(longest_path_to_node(node) for node in end_nodes)

    return {"Time":max_execution_time, "edge_hop":edge_hop, "time_node":time_node, "time_task":time_task}

def c1_time_limit(vector, problem):
    """
    Проверка на ограничение по времени
    """
    net_ststus = status(tuple(vector), problem)
    time = net_ststus['Time']
    return int(time <= problem.t_lim)

def c2_restriction_first_node(vector, problem):
    """
    Проверка на ограничение на первый узел 
    """
    net_ststus = status(tuple(vector), problem)
    time_node = net_ststus['time_node']
    return int(time_node[0] <= 1)

def f1_min_hop(vector, problem):
    net_ststus = status(tuple(vector), problem)
    edge_hop = net_ststus['edge_hop']
    return sum(edge_hop.values())

def f2_min_workload(vector, problem):
    net_ststus = status(tuple(vector), problem)
    t = net_ststus["Time"]
    time_node = np.array(net_ststus["time_node"])
    return sum(time_node/t)

def f2(vector, problem):
    """
    Вычисляет количество пересылок данных, необходимых при распределении задач
    по узлам сети, если изначально все задачи были на узле 0.
    
    Args:
        vector (list): Вектор распределения задач на узлы сети.
        problem (): network_graph (nx.Graph): Граф сети.
        
    Returns:
        int: Общее количество пересылок данных.
    """
    total_transfers = 0
    network_graph = problem.network_graph.graph

    for task_id, node_id in enumerate(vector):
        if node_id != 0:
            # Найти кратчайший путь от узла 0 до текущего узла
            path = nx.shortest_path(network_graph, source=0, target=node_id)
            
            # Увеличить счетчик пересылок на количество узлов в пути
            total_transfers += len(path) - 1
    
    return total_transfers + 1


In [4]:
from optiLib import NetworkOptimizationProblem

problem = NetworkOptimizationProblem(net, task, f_objective=[f1_min_hop, f2_min_workload], f_constraints=[c1_time_limit, c2_restriction_first_node], t_lim=5, net_speed=5000, bounds={0: [0,0]})

In [5]:
RANDOM_SEED = 1
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

from optiLib.optimizers import RandomSearchOptimizer, ParticleSwarmOptimizer

# Инициализация оптимизатора
optimizer = ParticleSwarmOptimizer(problem, inertia = 1.7,cognitive = 2.5 , social = 1.5, num_particles=100, iterations=1000)

# Запуск оптимизации
best_solution, best_value = optimizer.optimize()

from optiLib.helpers import print_solution
print_solution(best_solution, best_value)

for i, item in enumerate(optimizer.history):
    optimizer.history[i]= {'Эксперимент': 1, **item, **status(tuple(item['vector']), problem)}


optimizer.save(1)

Optimizing:   0%|          | 4/100000 [00:00<54:12, 30.74it/s]

[0 4 0 0 3 2 4 3 4 2 3 2 5]
[0 2 0 0 6 0 6 4 6 0 1 5 7]
[0 2 0 0 4 0 4 4 3 0 2 5 3]
[0 4 0 0 0 2 0 3 0 0 6 1 0]
[0 6 0 0 0 7 0 1 0 0 8 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 5 0 0 0 0 0]


Optimizing:   0%|          | 43/100000 [00:00<13:50, 120.42it/s]

[0 4 0 0 0 0 0 5 0 0 7 0 0]
[0 6 0 0 0 0 0 0 0 0 8 0 0]
[0 4 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 7 0 0 0 0 0 0 0 0 0 0 0]
[0 8 0 0 0 0 0 0 0 0 0 0 0]
[0 3 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0

c:\Users\user\Documents\MATH\optiLib\optimizers.py:107: RuntimeWarning: invalid value encountered in cast
  self.position += self.velocity.astype(self.problem.dtype)
Optimizing:   0%|          | 167/100000 [00:00<05:03, 328.71it/s]

...

Optimizing:   1%|          | 953/100000 [00:02<02:38, 626.80it/s]

[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0

Optimizing:   1%|          | 1016/100000 [00:02<03:36, 457.89it/s]


[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0

KeyboardInterrupt: 

In [11]:
problem.evaluate([0,2,2,2,1,1,0,0,0,0,0,0,0])

np.float64(21.26027397260274)

In [23]:
status(tuple([0,8,5,2,1,1,0,0,7,0,0,0,8]), problem)

{'Time': 3.4099999999999997,
 'edge_hop': {'01': 4,
  '02': 3,
  '03': 2,
  '14': 3,
  '25': 2,
  '36': 2,
  '47': 1,
  '58': 2,
  '69': 0,
  '710': 0,
  '711': 0,
  '811': 3,
  '98': 3,
  '1012': 4,
  '1112': 4},
 'time_node': [0.32000000000000006,
  2.0,
  0.5,
  0,
  0,
  0.5,
  0,
  0.13333333333333333,
  1.1],
 'time_task': [0.1,
  1.0,
  0.5,
  0.5,
  1.0,
  1.0,
  0.1,
  0.04,
  0.13333333333333333,
  0.04,
  0.02,
  0.02,
  0.1]}

In [10]:
RANDOM_SEED = 1
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

from optiLib.optimizers import RandomSearchOptimizer

# Инициализация оптимизатора
optimizer = RandomSearchOptimizer(problem,iterations = 1000)

# Запуск оптимизации
best_solution, best_value = optimizer.optimize()

from optiLib.helpers import print_solution
print_solution(best_solution, best_value)

for i, item in enumerate(optimizer.history):
    optimizer.history[i]={**item, **network_status_calculation(tuple(item['vector']), problem)}

optimizer.save()

Optimizing: 100%|██████████| 100000/100000 [00:06<00:00, 15017.17it/s]


Лучшее решение: [45 45 45 45 45 45 45 45 45 45 45 45 45 45 45 45]
Значение целевой функции: 6654266.181818182


In [11]:
RANDOM_SEED = 1
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

from optiLib.optimizers import GreedyOptimizer

# Инициализация оптимизатора
optimizer = GreedyOptimizer(problem)

# Запуск оптимизации
best_solution, best_value = optimizer.optimize()

from optiLib.helpers import print_solution
print_solution(best_solution, best_value)

for i, item in enumerate(optimizer.history):
    optimizer.history[i]={**item, **network_status_calculation(tuple(item['vector']), problem)}

optimizer.save()

Optimizing: 100%|██████████| 16/16 [00:00<00:00, 27.26it/s]


Лучшее решение: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Значение целевой функции: inf
